In [2]:
import math
from scripts_stock.cfg.out_file_name import OutFileName
from scripts_stock.cfg.set_dir import ProjectDir
from scripts_stock.cfg.stock_list import *
from scripts_stock.utils.analysis.stock_stat_index import df_to_stock_df, stock_kdj
import os
from scripts_stock.utils.common import CommonScript
from scripts_stock.utils.string_process import StringProcess
from scripts_stock.data_base.get_table_info import GetDataFromDB



In [ ]:

def get_weekly_price(input_df: pd.DataFrame):
    df1 = input_df
    df2 = df1.groupby(lambda x:math.floor(x/5)).min()
    df3 = df1.groupby(lambda x:math.floor(x/5)).max()
    df4 = df1.groupby(lambda x:math.floor(x/5)).last()
    df5 = df1.groupby(lambda x:math.floor(x/5)).first()
    df2["high"]=df3["high"]
    df2["close"] = df4["close"]
    df2["open"] = df5["open"]
    df2['stock_date'] = df3["date"]
    return df2

In [13]:
def get_stock_indicator(stock_df_in, if_print=False):
    stock_df = df_to_stock_df(
        stock_df_in[["open", "close", "high", "low", "stock_index", "stock_date"]])
    stock_kdj_ind, _ = stock_kdj(stock_df)
    if if_print:
        print(stock_kdj_ind)
    # stock_kdj_ind["stock_index"] = stock_df_in["stock_index"].values[0]
    return stock_kdj_ind

In [14]:
df_raw = GetDataFromDB.get_fuquan_all_one_stock_df(601398)
df_raw
start_from = df_raw.shape[0]%5
df1 = df_raw.iloc[start_from:].copy(deep=False).sort_values("date").reset_index()
df3 = get_weekly_price(df1)
get_stock_indicator(df3)

NOTE: Behavior of MACDH calculation has changed as of July 2017 - it is now 1/2 of previous calculated values
/home/davidyu/software/Anaconda/lib/python3.7/site-packages/stockstats.py:467: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  lambda x: np.fabs(x - x.mean()).mean())
/home/davidyu/software/Anaconda/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,date,open,close,high,low,stock_index,date_time,stock_date,macdh,cci,...,rsi_24,kdjk,kdjd,kdjj,boll_ub,boll_lb,macd,macds,wr_6,wr_10
0,2012-08-01,0.52,0.62,0.63,0.51,601398,2012-08-01,2012-08-01,0.000000,NaN,...,NaN,63.888889,54.629630,82.407407,NaN,NaN,0.000000,0.000000,8.333333,8.333333
1,2012-08-08,0.61,0.63,0.67,0.57,601398,2012-08-08,2012-08-08,0.000100,66.666667,...,100.000000,67.592593,58.950617,84.876543,0.639142,0.610858,0.000224,0.000125,25.000000,25.000000
2,2012-08-15,0.63,0.65,0.71,0.60,601398,2012-08-15,2012-08-15,0.000461,93.548387,...,100.000000,68.395062,62.098765,80.987654,0.663884,0.602783,0.000906,0.000445,30.000000,30.000000
3,2012-08-22,0.65,0.68,0.71,0.61,601398,2012-08-22,2012-08-22,0.001212,82.828283,...,100.000000,73.930041,66.042524,89.705075,0.697915,0.592085,0.002277,0.001066,15.000000,15.000000
4,2012-08-29,0.67,0.69,0.70,0.63,601398,2012-08-29,2012-08-29,0.001620,76.323988,...,100.000000,79.286694,70.457247,96.945588,0.714992,0.593008,0.003372,0.001752,10.000000,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,2024-11-12,6.13,6.00,6.17,6.00,601398,2024-11-12,2024-11-12,-0.024890,9.501436,...,59.852952,51.765979,53.399170,48.499598,6.520099,5.282901,0.245803,0.270693,79.518072,54.545455
598,2024-11-19,6.00,6.15,6.24,5.98,601398,2024-11-19,2024-11-19,-0.024473,30.477390,...,61.793307,53.794399,53.530913,54.321372,6.480568,5.417432,0.240102,0.264575,67.307692,42.148760
599,2024-11-26,6.13,6.16,6.18,6.05,601398,2024-11-26,2024-11-26,-0.024702,28.553837,...,61.921334,53.905746,53.655857,54.405524,6.456446,5.518554,0.233697,0.258400,57.142857,41.322314
600,2024-12-03,6.15,6.21,6.23,6.10,601398,2024-12-03,2024-12-03,-0.022716,46.081505,...,62.575623,51.198208,52.836641,47.921343,6.445449,5.598551,0.230005,0.252721,11.538462,41.284404


In [11]:
df1

,index,date,open,close,high,low,transaction_volume,stock_index,update_time
0,0,2012-07-26,0.52,0.52,0.53,0.51,5.768488e+07,601398,2024-12-11 02:09:05.003376
1,1,2012-07-27,0.53,0.57,0.57,0.52,1.085801e+08,601398,2024-12-11 02:09:05.003376
2,2,2012-07-30,0.57,0.56,0.60,0.55,9.346376e+07,601398,2024-12-11 02:09:05.003376
3,3,2012-07-31,0.57,0.60,0.61,0.56,8.729492e+07,601398,2024-12-11 02:09:05.003376
4,4,2012-08-01,0.60,0.62,0.63,0.59,9.397817e+07,601398,2024-12-11 02:09:05.003376
...,...,...,...,...,...,...,...,...,...
3005,3005,2024-12-04,6.21,6.26,6.28,6.17,2.228037e+09,601398,2024-12-11 02:09:05.003376
3006,3006,2024-12-05,6.25,6.26,6.33,6.23,1.659231e+09,601398,2024-12-11 02:09:05.003376
3007,3007,2024-12-06,6.26,6.32,6.34,6.25,1.968099e+09,601398,2024-12-11 02:09:05.003376
3008,3008,2024-12-09,6.30,6.32,6.37,6.28,1.982869e+09,601398,2024-12-11 02:09:05.003376


In [7]:

def get_data_sql_str_macd():
    return f"""SELECT *  FROM t_stock_kdj_daily_last """

# t_stock_kdj_weekly_last
conn = CommonScript.connect_to_db("test.db")
cursor = conn.cursor()
df1 = pd.read_sql_query(get_data_sql_str_macd(),conn)
print(df1)

           date    open   close    high     low  stock_index   date_time  \
0    2024-11-22    7.22    6.99    7.26    6.97       600011  2024-11-22   
1    2024-11-22   24.94   24.23   24.98   24.22          895  2024-11-22   
2    2024-11-22    1.95    1.94    2.00    1.90       600010  2024-11-22   
3    2024-11-22   57.06   56.01   57.19   56.01       600660  2024-11-22   
4    2024-11-22    6.50    6.42    6.60    6.40       601728  2024-11-22   
..          ...     ...     ...     ...     ...          ...         ...   
246  2024-11-22    6.26    6.13    6.35    6.13          877  2024-11-22   
247  2024-11-22    7.92    7.50    7.93    7.48       600061  2024-11-22   
248  2024-11-22  141.64  136.29  142.00  136.18          568  2024-11-22   
249  2024-11-22   41.05   38.92   41.50   38.83       600584  2024-11-22   
250  2024-11-22   42.12   41.08   43.88   41.00         2460  2024-11-22   

     stock_date  macdh      cci  ...    kdjk    kdjd    kdjj  boll_ub  \
0    2024-11-2

In [10]:
df1["stock_index"] =df1["stock_index"].astype(str)

In [12]:
pipeline = pyg.pipeline()

AttributeError: module 'pygwalker' has no attribute 'pipeline'

In [11]:
import pygwalker as pyg
gwalker = pyg.walk(df1)   

Box(children=(HTML(value='\n<div id="ifr-pyg-000627900250cc7akc4GAJWMZlbgsoFh" style="height: auto">\n    <hea…